In [1]:
cd /home

/home


In [2]:
from src.models import get_ASTROMER, build_input
from src.data import pretraining_pipeline
from tensorflow.keras.optimizers import Adam

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
window_size = 200

astromer =  get_ASTROMER(num_layers=2,
                         d_model=16,
                         num_heads=2,
                         dff=128,
                         base=1000,
                         dropout=.1,
                         maxlen=window_size,
                         pe_c=1.)

2023-03-27 14:39:56.536966: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-27 14:39:56.537000: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 7c62fa175994
2023-03-27 14:39:56.537008: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 7c62fa175994
2023-03-27 14:39:56.537105: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.32.3
2023-03-27 14:39:56.537120: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.32.3
2023-03-27 14:39:56.537125: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.32.3
2023-03-27 14:39:56.537389: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruct

In [4]:
optimizer = Adam(1e-3, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
astromer.compile(optimizer=optimizer)

In [5]:
data_dir = './data/records/alcock/fold_0/alcock_20/'
data = dict()
for subset in ['train', 'val']:
    data[subset] = pretraining_pipeline(os.path.join(data_dir, subset), 
                                        16, window_size, .2, .1, .1,
                                        sampling=True, shuffle=True, repeat=1, num_cls=None,
                                        normalize=True, cache=True)

[INFO] Repeating dataset x1 times
[INFO] Repeating dataset x1 times


In [6]:
hist = astromer.fit(data['train'], epochs=1, validation_data=data['val'])

6/6 [==============================] - 3s 223ms/step - loss: 1.1924 - r_square: -31.3705 - val_loss: 0.9357 - val_r_square: -9.3223


In [15]:
import tensorflow as tf 

min_index = tf.argmin(hist.history['val_loss'])
{
 'train_loss':hist.history['loss'][min_index],
'train_r2':hist.history['r_square'][min_index],
'val_loss':hist.history['val_loss'][min_index],
'val_r2':hist.history['val_r_square'][min_index]   
}


{'train_loss': 1.2357028722763062,
 'train_r2': -52.94478225708008,
 'val_loss': 0.9357174038887024,
 'val_r2': -9.322321891784668}

In [13]:
hist.history

{'loss': [1.2357028722763062],
 'r_square': [-52.94478225708008],
 'val_loss': [0.9357174038887024],
 'val_r_square': [-9.322321891784668]}